In [28]:
import pandas as pd
import os
import crew_optimizer
import openai
import json

In [29]:
from dotenv import load_dotenv
load_dotenv(override=True)

True

In [30]:
assignment = pd.read_json("assignments/assignment.json")
assignment = assignment['assignment'][4:254]
assignment

FR778    {'captain': 'C26', 'first_officer': 'FO12', 'd...
FR432    {'captain': 'C8', 'first_officer': 'FO2', 'dea...
FR272    {'captain': 'C43', 'first_officer': 'FO37', 'd...
FR933    {'captain': 'C40', 'first_officer': 'FO16', 'd...
FR381    {'captain': None, 'first_officer': None, 'dead...
                               ...                        
C23                                                    NaN
C24                                                    NaN
C25                                                    NaN
C26                                                    NaN
C27                                                    NaN
Name: assignment, Length: 250, dtype: object

In [ ]:
# Convert assignment to string for LLM analysis
# Convert pandas Series to dict to make it JSON serializable
assignment_dict = assignment.to_dict()
assignment_text = json.dumps(assignment_dict, indent=2)
print("Assignment converted to JSON string for LLM analysis")
print(f"Text length: {len(assignment_text)} characters")
print(f"Text lines: {len(assignment_text.split(chr(10)))} lines")
print("\nFirst 500 characters:")
print("=" * 50)
print(assignment_text[:500])
print("..." if len(assignment_text) > 500 else "")

TypeError: Object of type Series is not JSON serializable

In [ ]:
# Store the complete assignment JSON string for LLM analysis
print("COMPLETE ASSIGNMENT JSON FOR LLM:")
print("=" * 60)
print(assignment_text)
print("=" * 60)
print("JSON string is ready for LLM analysis to suggest new destroy/repair operators")

COMPLETE ASSIGNMENT TEXT FOR LLM:
CREW ASSIGNMENT ANALYSIS DATA

SUMMARY STATISTICS:
- Total flights: 250
- Flights with assigned captains: 249
- Flights with assigned first officers: 244
- Unassigned flights: 6

CREW WORKLOAD DISTRIBUTION:
Captains:
- Active captains: 48
- Max flights per captain: 13
- Min flights per captain: 1
- Average flights per captain: 5.2
- Overworked captains (>4 flights): 25
First Officers:
- Active first officers: 49
- Max flights per first officer: 12
- Min flights per first officer: 1
- Average flights per first officer: 5.0
- Overworked first officers (>4 flights): 25

ASSIGNMENT PATTERNS:
- Flights missing captains: 1
  Examples: D1_F022
- Flights missing first officers: 6
  Examples: D2_F039, D2_F032, D2_F006, D3_F018, D3_F009

SAMPLE ASSIGNMENTS (first 10 flights):
- D1_F026: Captain=C26, FO=FO20
- D1_F043: Captain=C44, FO=FO35
- D1_F005: Captain=C34, FO=FO46
- D1_F023: Captain=C6, FO=FO28
- D1_F031: Captain=C32, FO=FO32
- D1_F014: Captain=C14, FO=FO2

In [ ]:
openai_api_key = os.getenv('OPENAI_API_KEY')
print(openai_api_key[:-4])

NameError: name 'getenv' is not defined